### Постановка проблемы

В данной лабораторной работе будер рассмотрена проблема медленного кода и как его ускорить с помощью "профилирования", для начала рассмотрим пример медленного кода

In [ ]:
from decimal import *

In [ ]:
# slow_program.py

def exp(x):
    getcontext().prec += 2
    i, lasts, s, fact, num = 0, 0, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 1
        fact *= i
        num *= x
        s += num / fact
    getcontext().prec -= 2
    return +s

exp(Decimal(150))
exp(Decimal(400))
exp(Decimal(3000))


Decimal('7.646200989054704889310727660E+1302')

### Получение данных

In [1]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!git clone https://github.com/Romiono/BD_lab1

Cloning into 'BD_lab1'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.
Resolving deltas: 100% (3/3), done.


### Самый легкий способ профилирования
Самым простым способом профилирования является испольнование команды Unix time



In [3]:
!time python BD_lab1/slow_program.py


real	0m16.053s
user	0m15.433s
sys	0m0.114s


### Точные инструменты профилирования

Существуют более сложные инструменты профилирования, например cProfile, который дает очень много сведений, slow_program.py запустим его с помощью cProfile

In [4]:
!python -m cProfile -s cumulative BD_lab1/slow_program.py

         913 function calls (892 primitive calls) in 15.014 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      3/1    0.000    0.000   15.014   15.014 {built-in method builtins.exec}
        1    0.000    0.000   15.014   15.014 slow_program.py:1(<module>)
        3   15.010    5.003   15.010    5.003 slow_program.py:3(exp)
      3/1    0.000    0.000    0.003    0.003 <frozen importlib._bootstrap>:1022(_find_and_load)
      3/1    0.000    0.000    0.003    0.003 <frozen importlib._bootstrap>:987(_find_and_load_unlocked)
      3/1    0.000    0.000    0.003    0.003 <frozen importlib._bootstrap>:664(_load_unlocked)
      2/1    0.000    0.000    0.003    0.003 <frozen importlib._bootstrap_external>:877(exec_module)
      4/1    0.000    0.000    0.003    0.003 <frozen importlib._bootstrap>:233(_call_with_frames_removed)
        1    0.000    0.000    0.003    0.003 decimal.py:1(<module>)
      3/2    0.000    0.000   

### Вывод относительно программы slow_program.py
Проанализировав вывод Cprofile можно понять, что причиной медленной работы является функция exp()

Напишем декаротар считающий время для функций

In [ ]:
from decimal import Decimal, getcontext
from functools import wraps
from time import perf_counter, process_time

In [ ]:
def timeit_wrapper(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = process_time()  # Используем process_time для измерения времени процессора
        func_return_val = func(*args, **kwargs)
        end = process_time()
        print('{0:<10}.{1:<8} : {2:<8.6f}'.format(func.__module__, func.__name__, end - start))
        return func_return_val
    return wrapper

Перепишем нашу функцию slow_program.py с использованием этого декоратора

In [ ]:
@timeit_wrapper
def exp_tracking(x):
    getcontext().prec += 2
    i, lasts, s, fact, num = 0, 0, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 1
        fact *= i
        num *= x
        s += num / fact
    getcontext().prec -= 2
    return +s

# Печать заголовков для результата
print('{0:<10} {1:<8} {2:^8}'.format('module', 'function', 'time'))

# Тестовый вызов функции
exp_tracking(Decimal(150))
exp_tracking(Decimal(400))
exp_tracking(Decimal(3000))

module     function   time  
__main__  .exp_tracking : 0.005099
__main__  .exp_tracking : 0.049364
__main__  .exp_tracking : 15.004622


Decimal('7.646200989054704889310727660E+1302')

### Ускорение кода
Рассмотрим методы ускорения кода:
1. ИСПОЛЬЗУЙТЕ ВСТРОЕННЫЕ ТИПЫ ДАННЫХ
2. ПРИМЕНЯЙТЕ КЭШИРОВАНИЕ (МЕМОИЗАЦИЮ) С ПОМОЩЬЮ LRU_CACHE
3. ИСПОЛЬЗУЙТЕ ЛОКАЛЬНЫЕ ПЕРЕМЕННЫЕ
4. ОБОРАЧИВАЙТЕ КОД В ФУНКЦИИ
5. НЕ ОБРАЩАЙТЕСЬ К АТРИБУТАМ
6. ОСТЕРЕГАЙТЕСЬ СТРОК
7. ИСПОЛЬЗУЮТЕ ГЕНЕРАТОРЫ


### Кеширование

Рассмотрим работу с кешированием, в данно случае при втором вызове функции slow_func() результат будет выдан мгновенно, так как он уже кеширован с помощью @functools.lru_cache()

In [ ]:
import functools
import time


In [ ]:
# кэширование до 12 различных результатов
@functools.lru_cache(maxsize=12)
def slow_func(x):
    time.sleep(2)  # Имитируем длительные вычисления
    return x

slow_func(1)  # ... ждём 2 секунды до возврата результата
slow_func(1)  # результат уже кэширован - он возвращается немедленно!

slow_func(3)  # ... опять ждём 2 секунды до возврата результата


3

### Использование локальных переменных

Теперь рассмотрим примеры кода c ускореннием с помощью локальных переменных

In [5]:
#  Пример #1
class FastClass:

    def do_stuff(self):
        temp = self.value  # это ускорит цикл
        for i in range(10000):
          temp = i # Выполняем тут некие операции с `temp`

#  Пример #2
import random

def fast_function():
    r = random.random
    for i in range(10000):
        print(r())  # здесь вызов `r()` быстрее, чем был бы вызов random.random()


### Использование функций

Здесь все просто, нужно использовать функции, включая глобальную функцию main(), для ускорения кода

In [ ]:
def main():
    ...  # Весь код, который раньше был глобальным

main()


### Отсутствие обращения к атрибутам
В некоторых случаях оператор "." может замедлить выполнения программы, рассмотрим на примерах

In [ ]:
#  Медленно:
import re

def slow_func():
    for i in range(10000):
        re.findall(regex, line)  # Медленно!

#  Быстро:
from re import findall

def fast_func():
    for i in range(10000):
        findall(regex, line)  # Быстрее!


### Испльзование строк
Их использование может очень сильно замедлить выполнение программы. В частности, речь идёт о форматировании строк с использованием %s и .format()

In [ ]:
from string import Template


In [ ]:
s = "gdfgdf"
t = "gfsgsdfgdfsgsdfgsdfg"

f'{s} {t}'  # Быстро!
s + '  ' + t
' '.join((s, t))
'%s %s' % (s, t)
'{} {}'.format(s, t)
Template('$s $t').substitute(s=s, t=t)  # Медленно!


'gdfgdf gfsgsdfgdfsgsdfgsdfg'

### py-spy
Существую ситуации, когда локальная отладка кода недоступна, тогда следует использовать пакет py-spy. Это — профилировщик, способный исследовать программы, которые уже запущены. Допустим, что slow_program.py это программа, которая выполняется длительное время, тогда ее вызов будет выглядеть вот так, но к сожалению в google colab нельзя получить id процесса, с помощью "ps" или других подобных инструментов.

In [6]:
!pip install py-spy
!python BD_lab1/slow_program.py &
![1] 1129587
!ps -A -o pid,cmd | grep python
...
!1129587 python BD_lab1/slow_program.py
!1130365 grep python
!sudo env "PATH=$PATH" py-spy top --pid 1129587


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.6 MB/s eta 0:00:00
/bin/bash: line 1: [1]: command not found
     67 [python3] <defunct>
     68 python3 /usr/local/bin/colab-fileshim.py
     89 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 
    415 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-7cd24
    450 /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/debugpy/adapter --for-server 50103 
   1054 /bin/bash -c ps -A -o pid,cmd | grep python
   1056 grep python
/bin/bash: line 1: 1129587: command not found
/bin/bash: line 1: 1130365: command not found
Error: Failed to get process executable name. Check that the process is running.
Reason: No such file or directory (os error 2)
Reason: No such file or directory (os error 2)


### Более глубокое исследование кода
Если рассмотренные выше профилировщики не помогли решить проблему, то можно использовать line_profiler, он может помочь выяснить сколько выполняется каждая строка, рассмотрим на примере slow_program.py

In [9]:
!pip install line_profiler
!kernprof -l -v BD_lab1/slow_program_line_profile.py

Wrote profile results to slow_program_line_profile.py.lprof
Timer unit: 1e-06 s

Total time: 16.2094 s
File: BD_lab1/slow_program_line_profile.py
Function: exp at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           @profile
     5                                           def exp(x):
     6         3          5.9      2.0      0.0      getcontext().prec += 2
     7         3          1.5      0.5      0.0      i, lasts, s, fact, num = 0, 0, 1, 1, 1
     8      4605       5452.1      1.2      0.0      while s != lasts:
     9      4602       1410.8      0.3      0.0          lasts = s
    10      4602       3545.6      0.8      0.0          i += 1
    11      4602       9331.6      2.0      0.1          fact *= i
    12      4602       3769.9      0.8      0.0          num *= x
    13      4602   16185833.2   3517.1     99.9          s += num / fact
    14         3         24.3      8.1      0.0      getcontext().prec 

### Профилирование ОЗУ
Для профилирования ОЗУ нам поможет memory_profiler

In [10]:
!pip install memory_profiler psutil
!python -m memory_profiler BD_lab1/slow_program_with_profile.py


Filename: BD_lab1/slow_program_with_profile.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     4     38.1 MiB     38.1 MiB           1   @profile
     5                                         def exp(x):
     6     38.1 MiB      0.0 MiB           1       getcontext().prec += 2
     7     38.1 MiB      0.0 MiB           1       i, lasts, s, fact, num = 0, 0, 1, 1, 1
     8     38.1 MiB      0.0 MiB         310       while s != lasts:
     9     38.1 MiB      0.0 MiB         309           lasts = s
    10     38.1 MiB      0.0 MiB         309           i += 1
    11     38.1 MiB      0.0 MiB         309           fact *= i
    12     38.1 MiB      0.0 MiB         309           num *= x
    13     38.1 MiB      0.0 MiB         309           s += num / fact
    14     38.1 MiB      0.0 MiB           1       getcontext().prec -= 2
    15     38.1 MiB      0.0 MiB           1       return +s


Filename: BD_lab1/slow_program_with_profile.py

Line #    Mem usage    Increme

### Другие способы оптимизации
Также существую иные способы оптимизации, например PyPy - это пакет, который способен перекомпилировать python в С

In [11]:
!apt-get install pypy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pypy-lib
Suggested packages:
  pypy-doc pypy-tk
The following NEW packages will be installed:
  pypy pypy-lib
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 16.2 MB of archives.
After this operation, 88.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pypy-lib amd64 7.3.9+dfsg-1 [2,484 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pypy amd64 7.3.9+dfsg-1 [13.7 MB]
Fetched 16.2 MB in 1s (30.9 MB/s)
Selecting previously unselected package pypy-lib:amd64.
(Reading database ... 123635 files and directories currently installed.)
Preparing to unpack .../pypy-lib_7.3.9+dfsg-1_amd64.deb ...
Unpacking pypy-lib:amd64 (7.3.9+dfsg-1) ...
Setting up pypy-lib:amd64 (7.3.9+dfsg-1) ...
Selecting previously unselected package pypy.
(Reading database ...

In [12]:
!time python BD_lab1/slow_program.py
!time pypy BD_lab1/slow_program.py


real	0m15.269s
user	0m15.022s
sys	0m0.017s

real	0m3.874s
user	0m3.679s
sys	0m0.051s


### Проблемы PyPY
Этот инструмент поддерживает проекты, нуждающиеся в C-привязках, такие, как numpy, но это создаёт значительную дополнительную нагрузку на систему, что сильно замедляет соответствующие библиотеки, сводя на нет другие улучшения производительности. PyPy, кроме того, не решает проблем с производительностью в ситуациях, когда применяются внешние библиотеки, или в случаях, когда речь идёт о работе с базами данных. И, аналогично, если речь идёт о программах, производительность которых привязана к подсистеме ввода/вывода, не стоит ожидать значительной выгоды от применения PyPy.
Существуют другие решения, если вы готовы идти на компромисы такие как prometeo